## FORKED FROM https://github.com/bella-mir/geocode-mapbox-api/blob/main/fun_geocode.ipynb

# GEOCODE AND FUN

In [1]:
pip install geopandas

In [2]:
# import libraries
import pandas as pd
import geopandas as gpd

# For working with requests
import requests
import json


### Defining parameters for geocoding ###

___1. (!!!) Insert your API key:___

In [1]:
API_KEY = ###

2. Trying with a sample address:

In [4]:
sample_address = 'Москва, Волхонка 11'
sample_address = sample_address.replace(' ', '+')
url = f'https://geocode-maps.yandex.ru/1.x/?apikey={API_KEY}&geocode={sample_address}&format=json'

3. Creating a get request

In [5]:
response = requests.get(url)

4. Checking response code

In [6]:
print(response)

<Response [200]>


5. Saving response data to a variable and extracting coordinates

In [7]:
data = response.json()
coordinates = data['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'].split()
longitude, latitude = coordinates

### Geocoding addresses from a csv file and saving them ###

1. Reading csv

In [12]:
df = pd.read_csv('./alex_schools.csv', sep = ',')

2. Checking the data

In [13]:
df.head()

,id,type,name,address,area,series,capacity
0,1,детский сад,Школа № 1503. Дошкольное отделение,"Москва, улица Константинова, 20, корп. 1",857,индивидуальный проект,NaN
1,2,детский сад,"Школа № 1503, дошкольное отделение","Москва, улица Павла Корчагина, 12",1842,VI-49,NaN
2,3,детский сад,Школа № 1503. Дошкольное отделение,"Москва, улица Космонавтов, 14, корп. 4",2167,2Мг-04-3,NaN
3,4,детский сад,Солнышко АНО ДОО Мир Детства,"Москва, улица Кибальчича, 8, стр. 1",1244,VI-49,NaN
4,5,детский сад,Школа № 1503. Дошкольное отделение,"Москва, улица Павла Корчагина, 1, стр. 1",2111,2Мг-04-3,NaN


3. Adding columns for coordinates

In [16]:
df['longitude'] = ''
df['latitude'] = ''

4. Geocoding!

In [17]:
for index, row in df.iterrows():
    address = row['address']
    url = f'https://geocode-maps.yandex.ru/1.x/?apikey={API_KEY}&geocode={address}&format=json'
    try:
        response = requests.get(url)
        data = response.json()
        coordinates = data['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'].split()
        longitude, latitude = coordinates
        df.at[index, 'longitude'] = longitude
        df.at[index, 'latitude'] = latitude
    except Exception as e:
        print(f"Error with {address}: {e}")


5. Saving the data into csv

In [18]:
df.to_csv('./alex_schools_yandex_geocoded.csv', index=False, index_label=False, encoding='utf-8')

6. Creating Geopandas object

In [19]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326")


7. Exploring the data

In [20]:
import folium
gdf.explore()

8. Saving the data into Geopackage

In [21]:
gdf.to_file('alex_schools.gpkg', driver='GPKG', layer='alex_schools')